In [1]:
import pandas as pd

sales_data = pd.read_csv('cleaned.csv')

In [2]:
sales_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20


In [3]:
sales_data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Revenue'],
      dtype='object')

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
description_words = tfidf.fit_transform(sales_data['Description'].astype(str))


In [5]:
from sklearn.cluster import KMeans
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters,random_state=42)
sales_data['CategoryCluster'] = kmeans.fit_predict(description_words)

In [6]:
import numpy as np
terms = np.array(tfidf.get_feature_names_out())
order_centroids = kmeans.cluster_centers_.argsort()[:,::-1]
cluster_labels = {}
for i in range(n_clusters):
    top_terms = terms[order_centroids[i, :3]]  
    cluster_labels[i] = " ".join(top_terms)
    
sales_data['Category'] = sales_data['CategoryCluster'].map(cluster_labels)

In [7]:
sales_data['InvoiceDate'] = pd.to_datetime(sales_data['InvoiceDate'])

sales_data['Year'] = sales_data['InvoiceDate'].dt.year
sales_data['Month'] = sales_data['InvoiceDate'].dt.month
sales_data['Day'] = sales_data['InvoiceDate'].dt.day
sales_data['Hour'] = sales_data['InvoiceDate'].dt.hour
sales_data['WeekDay'] = sales_data['InvoiceDate'].dt.weekday
sales_data['Week'] = sales_data['InvoiceDate'].dt.isocalendar().week

sales_data.to_csv('Transactions.csv',index=False)

In [8]:
import datetime
reference_date = sales_data['InvoiceDate'].max() + pd.Timedelta(days=1)

In [9]:
customer = sales_data.groupby('CustomerID').agg({
  'InvoiceDate':['max','nunique'],
  'InvoiceNo' : 'nunique',
  'Revenue': 'sum',
  'Quantity': 'sum'
})

customer.columns = ['LastPurchaseDate','UniquePurchaseDays','TotalOrders','TotalRevenue','TotalQuantity']
customer.reset_index(inplace=True)


In [11]:
customer['Recency'] = (reference_date - customer['LastPurchaseDate']).dt.days
customer['Frequency'] = customer['TotalOrders']
customer['Monetary'] = customer['TotalRevenue']

In [13]:
customer['AverageOrderValue'] = customer['TotalRevenue'] / customer['TotalOrders']
customer['PurchaseFrequency'] = customer['TotalOrders'] / customer['UniquePurchaseDays']
customer['CLV'] = customer['AverageOrderValue'] * customer['PurchaseFrequency'] * 12  # yearly CLV


customer['AverageBasketSize'] = customer['TotalQuantity'] / customer['TotalOrders']

customer.to_csv("customer_features.csv", index=False)